# Creating photonic circuits

Let's start by creating a very simple circuit, in pure python, by connecting a series of components from a PDK.

In [ ]:
import cspdk.si220
from cspdk.si220 import cells
import gdsfactory as gf

cspdk.si220.PDK.activate()

pdk = gf.get_active_pdk()

In [ ]:
c = gf.Component()
s1 = c << cells.straight_sc(length=50)
s2 = c << cells.straight_sc(length=20)
b1 = c << cells.bend_sc()
b1.connect('o1', s1.ports['o2'])
s2.connect('o1', b1.ports['o2'])
c.add_port('o1', port=s1.ports['o1'])
c.add_port('o2', port=s2.ports['o2'])
c.rename('first_circuit')
c

Note that the circuit can now be serialized to a yaml format that captures both the connectivity (netlist) information and placements that would be required to rebuild the circuit.

In [ ]:
yaml_circuit = c.to_yaml()
print(yaml_circuit)

And the yaml netlist can also be deserialized to reinstantiate the gdsfactory component and back out to GDS.

In [ ]:
loaded = gf.read.from_yaml(yaml_circuit)
loaded

Of course, you could also edit the netlist in yaml and regenerate a different circuit. Let's try it.

In [ ]:
# first, let's write the netlist to file
c.write_netlist("first_circuit.pic.yml")
# now try adding another instance to the netlist. maybe another bend at the end

In [ ]:
gf.clear_cache()
gf.read.from_yaml('first_circuit.pic.yml')

## Routes

Until now, our circuits have been fairly simple. For more complicated circuits, we want to be able to easily define routes between components. There are various functions in gdsfactory which allow us to do this.

### Automatic bundle routing
In photonics, it is common to want route a group of waveguides together in what we call a "bundle". This is similar to what is often called a "bus" in electrical routing.

The `get_bundle` function can route a bundle between a list of input and corresponding output ports with a simple automatic routing routine.

In [ ]:
c2 = gf.Component()
mmi1 = c2.add_ref(cells.mmi1x2_sc(), "mmi1")
mmi2 = c2.add_ref(cells.mmi1x2_sc(), "mmi2")
mmi2.rotate(180, "o2")
mmi2.move("o2", mmi1.ports["o3"].center + (100, -50))
bundle = gf.routing.get_bundle([mmi1.ports["o2"], mmi1.ports["o3"]], 
                               [mmi2.ports["o3"], mmi2.ports["o2"]])
for route in bundle:
    c2.add(route.references)
c2

### Custom bundle routing
Sometimes we wish to route in a more custom fashion. Whether it be to route around obstacles, acheive a particular pathlength, etc., the `get_bundle_from_steps` function allows us to route in the sequence of directions specified by our `steps` directives.

In [ ]:
c2 = gf.Component()
mmi1 = c2.add_ref(cells.mmi1x2_sc(), "mmi1")
mmi2 = c2.add_ref(cells.mmi1x2_sc(), "mmi2")
mmi2.rotate(180, "o2")
mmi2.move("o2", mmi1.ports["o3"].center + (100, -50))
bundle = gf.routing.get_bundle_from_steps([mmi1.ports["o2"], mmi1.ports["o3"]], 
                                          [mmi2.ports["o3"], mmi2.ports["o2"]],
                                          steps=[
                                              {"dx": 20},
                                              {"dy": 100},
                                              {"dx": 20}
                                          ])
for route in bundle:
    c2.add(route.references)
c2

### All-angle routing
You are not constrained to route at manhattan angles. The `get_bundle_all_angle` router can be used to route bundles at any angle, with a similar syntax to the step router.

Here is an example with a simple, automatic, non-manhattan bundle.

In [ ]:
c2 = gf.Component()
mmi1 = c2.add_ref(cells.mmi1x2_sc(), "mmi1")
mmi2 = c2.add_ref(cells.mmi1x2_sc(), "mmi2")
mmi2.rotate(180, "o2")
mmi2.move("o2", mmi1.ports["o3"].center + (100, -50))
bundle = gf.routing.get_bundle_all_angle([mmi1.ports["o2"], mmi1.ports["o3"]], 
                                          [mmi2.ports["o3"], mmi2.ports["o2"]],
                                          connector="simple",
                                          )
for route in bundle:
    c2.add(route.references)
c2

And here we specify manual steps. Instead of specifying steps in x or y, we specify steps of size `ds` and the angle to exit the current step in. If you configure the Klayout ruler to display the angle, it makes it very handy to sketch this out once and transcribe as a series of steps with the ruler.

In [ ]:
gf.CONF.allow_offgrid = True

c2 = gf.Component()
mmi1 = c2.add_ref(cells.mmi1x2_sc(), "mmi1")
mmi2 = c2.add_ref(cells.mmi1x2_sc(), "mmi2")
mmi2.rotate(180, "o2")
mmi2.move("o2", mmi1.ports["o3"].center + (100, -50))
bundle = gf.routing.get_bundle_all_angle([mmi1.ports["o2"], mmi1.ports["o3"]], 
                                          [mmi2.ports["o3"], mmi2.ports["o2"]],
                                          connector="simple",
                                          steps=[
                                              {"ds": 20, "exit_angle": -20},
                                              {"ds": 50, "exit_angle": 180 + 20},
                                          ]
                                          )
for route in bundle:
    c2.add(route.references)
c2.write_gds("sample_offgrid.gds", flatten_offgrid_references=True)
c2

### Routes in yaml circuits
In yaml circuits, we may also define `routes`. See the example below

In [ ]:
# our PDK doesn't have the routing options we want baked in, so let's hack our way there
pdk.routing_strategies['get_bundle'] = gf.routing.get_bundle
pdk.routing_strategies['get_bundle_from_steps'] = gf.routing.get_bundle_from_steps
pdk.routing_strategies['all_angle'] = gf.routing.get_bundle_all_angle
# cell_from_yaml_template will create a proper cell definition, so let's clear the cache between each run of this notebook cell
gf.clear_cache()

c = gf.read.cell_from_yaml_template("circuits/photonic_circuit.pic.yml",
                                    name="sample_circuit")
c()

### Templated yaml cells
There are some benefits to defining your circuits as yaml. For instance,
- the declarative nature of the format makes your circuit architecture clear
- your circuit is already in a nice exchange format
- it is easy to update a circuit without reloading the python instance

But yaml is also much less flexible than pure python, making it harder to define programmatic cells with complex functionality. In an effort to get the best of both worlds, we also allow you to define your circuit as a jinja template.

In this case, `cell_from_yaml_template` creates a function from the `*.yml.j2` file, and the header to the file must be the `default_settings` which are the input arguments to the pcell. When calling the function, the template is evaluating, allowing you to insert logic such as for-loops and conditionals into the circuit, in additional to simple variables and expressions.

In [ ]:
c = gf.read.cell_from_yaml_template("circuits/photonic_circuit_pcell.pic.yml.j2",
                                    name="my_yaml_pcell")
c?

In [ ]:
c()

In [ ]:
dx500 = c(dx=500)
dx500

In [ ]:
dx500.write_netlist("dx500_extracted.pic.yml")

## Simulating the circuit with SAX
SAX is an open source S-matrix simulator. It is designed to interoperate well with gdsfactory and can take the extracted yaml output as an input circuit to perform an easy post-layout circuit validation.

In [ ]:
import cspdk.si220.models
import sax
from jax import jit
import jax.numpy as jnp
import matplotlib.pyplot as plt

netlist = dx500.get_netlist()
models = cspdk.si220.get_models()
circuit, info = sax.circuit(netlist=netlist,
                            models=models)
circuit?

Now that we have a circuit object, we can simulate it at a couple of different wavelengths.

In [ ]:
%time circuit(wl=1.500)

In [ ]:
%time circuit(wl=1.550)

Since sax is built on top of jax, we can use jax's jitting capabilities to dramatically speed up our circuit execution. There is always a bit of overhead on the first function execution to jit the function. But after that, subsequent executions are *extremely* fast.

In [ ]:
circuit_fast = jit(circuit)

In [ ]:
%time circuit_fast(wl=1.500)

In [ ]:
%time circuit_fast(wl=1.550)

Now, with our fast circuit implementation, we can quickly run 1000 simulations at different wavelenths and plot the results.

In [ ]:
wl = jnp.linspace(1.4, 1.7, 1000)
%time S = circuit(wl=wl)

In [ ]:
plt.plot(wl * 1e3, abs(S["o1", "o2"]) ** 2)
plt.ylim(-0.05, 1.05)
plt.xlabel("λ [nm]")
plt.ylabel("T")
plt.ylim(-0.05, 1.05)
plt.show()

### A more interesting example
Let's now repeat the above with a parametric circuit of significance: a simple Mach-Zehnder Interferometer.

This example is similar to the previous one, except we space the MMI's evenly and create a controlled length difference between the two arms. The dL between the arms creates a phase delay which causes interference at the output. Hence, by varying the dL between the arms we should be able to create an interesting frequency response.

In [ ]:
c = gf.read.cell_from_yaml_template("circuits/custom_mzi.pic.yml.j2",
                                    name="custom_mzi")
c()

In [ ]:
circuit_by_dL = {dL: c(dL=dL) for dL in [100, 200, 300]}
for dL, circuit_layout in circuit_by_dL.items():
    circuit_layout.plot()

In [ ]:
wl = jnp.linspace(1.545, 1.555, 1000)

for dx, circuit_layout in circuit_by_dL.items():
    circuit_netlist = circuit_layout.get_netlist()
    circuit_sim, _ = sax.circuit(circuit_netlist, models=models)
    fast_sim = jit(circuit_sim)
    results = circuit_sim(wl=wl)
    plt.plot(wl * 1e3, abs(results["o1", "o2"]) ** 2, label=f"dx = {dx}")
plt.ylim(-0.05, 1.05)
plt.xlabel("λ [nm]")
plt.ylabel("T")
plt.ylim(-0.05, 1.05)
plt.legend()
plt.show()

We engineered this example to generate very pronounced parasitic effects based on the shape of the routing. However, for a complex circuit, hopefully it is clear how valuable of a tool this can be to analyze and tune post-layout behaviour of the circuit.